

LIGHTER TRACKER - SISTEMA DE PREDIÇÃO DE PERDA DE ISQUEIROS
=============================================================================

* Projeto: Sistema de Machine Learning para prever perda de isqueiros
* Algoritmo: Regressão Logística (Classificação Binária)
* Target: perdeu (True/False)
* Objetivo: Fornecer recomendações aos usuários sobre probabilidade de perda

#### Autor: Murilo Souza, Cientista de Dados em Transição
============================================================================


# 1. IMPORTAÇÃO DAS BIBLIOTECAS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import random
import warnings
warnings.filterwarnings('ignore')

# Configurações de exibição
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# 2. GERAÇÃO DA BASE SINTÉTICA DE DADOS

In [2]:

def gerar_base_sintetica(n_registros=1200, seed=42):
    """
    Gera uma base sintética de dados para o sistema Lighter Tracker

    Parâmetros:
    - n_registros: Número de registros a serem gerados (padrão: 1200)
    - seed: Seed para reprodutibilidade (padrão: 42)

    Retorna:
    - DataFrame com dados sintéticos de uso de isqueiros
    """

    np.random.seed(seed)
    random.seed(seed)

    print(f"🔄 Gerando {n_registros} registros sintéticos...")

    # Definindo as opções categóricas baseadas no contexto real de uso
    opcoes = {
        'genero': ['M', 'F'],
        'marcas': ['Bic', 'Zippo', 'Clipper', 'Cricket', 'Ronson', 'Generico'],
        'cores': ['Azul', 'Preto', 'Branco', 'Vermelho', 'Verde', 'Amarelo', 'Prata', 'Dourado'],
        'contextos_uso': ['Casa', 'Rua', 'Trabalho', 'Bar', 'Parque', 'Social', 'Festa', 'Carro'],
        'propositos_uso': ['Cigarro', 'Tabaco', 'Vela', 'Incenso', 'Fogao', 'Camping', 'Charuto', 'Outro'],
        'perfil_fumante': ['Pesado', 'Moderado', 'Social', 'Esporadico', 'Nao_fumante']
    }

    dados = []

    # Data de início: 1 ano atrás
    data_inicio = datetime.now() - timedelta(days=365)

    for i in range(n_registros):
        # Características do usuário
        usuario_id = f"USER_{i+1:04d}"
        nome = f"Usuario_{i+1}"
        genero = np.random.choice(opcoes['genero'])
        idade = np.random.randint(18, 65)

        # Faixa etária baseada na idade
        if idade <= 25:
            faixa_idade = '18-25'
        elif idade <= 35:
            faixa_idade = '26-35'
        elif idade <= 50:
            faixa_idade = '36-50'
        else:
            faixa_idade = '51+'

        # Perfil de fumante (influencia no padrão de uso)
        perfil_fumante = np.random.choice(opcoes['perfil_fumante'],
                                        p=[0.15, 0.25, 0.30, 0.20, 0.10])

        # Características do isqueiro
        marca = np.random.choice(opcoes['marcas'],
                               p=[0.35, 0.15, 0.20, 0.15, 0.10, 0.05])  # Bic é mais comum
        cor = np.random.choice(opcoes['cores'])
        tipo_isqueiro = 'Descartavel' if marca in ['Bic', 'Cricket', 'Generico'] else 'Recarregavel'

        # Preço baseado na marca
        preco_base = {'Bic': 2.5, 'Zippo': 50.0, 'Clipper': 8.0,
                     'Cricket': 1.5, 'Ronson': 15.0, 'Generico': 1.0}
        preco_compra = preco_base[marca] + np.random.uniform(-0.5, 1.5)
        preco_compra = max(preco_compra, 0.5)  # Preço mínimo

        # Datas
        data_compra = data_inicio + timedelta(days=np.random.randint(0, 330))

        # Tempo de posse (dias desde a compra até uso/perda)
        # Isqueiros mais caros tendem a ser mantidos por mais tempo
        if marca in ['Zippo', 'Ronson']:
            tempo_posse = np.random.randint(30, 200)
        else:
            tempo_posse = np.random.randint(5, 120)

        data_uso = data_compra + timedelta(days=tempo_posse)

        # Padrões de uso baseados no perfil
        multiplicador_uso = {'Pesado': 3.0, 'Moderado': 2.0, 'Social': 1.5,
                           'Esporadico': 0.8, 'Nao_fumante': 0.3}

        # Número de ascendimentos baseado no tempo de posse e perfil
        ascendimentos_base = tempo_posse * multiplicador_uso[perfil_fumante]
        num_ascendimentos = int(max(1, np.random.poisson(ascendimentos_base)))

        # Frequência de uso (ascendimentos por dia)
        frequencia_uso = num_ascendimentos / max(tempo_posse, 1)

        # Contexto e propósito de uso (com lógica de probabilidade)
        # Fumantes pesados usam mais na rua, não fumantes mais em casa
        if perfil_fumante in ['Pesado', 'Moderado']:
            contexto_uso = np.random.choice(opcoes['contextos_uso'],
                                          p=[0.20, 0.25, 0.15, 0.15, 0.10, 0.10, 0.03, 0.02])
            proposito_uso = np.random.choice(opcoes['propositos_uso'],
                                           p=[0.60, 0.25, 0.05, 0.05, 0.02, 0.02, 0.01, 0.00])
        else:
            contexto_uso = np.random.choice(opcoes['contextos_uso'],
                                          p=[0.50, 0.10, 0.20, 0.05, 0.05, 0.05, 0.03, 0.02])
            proposito_uso = np.random.choice(opcoes['propositos_uso'],
                                           p=[0.20, 0.10, 0.25, 0.20, 0.15, 0.05, 0.03, 0.02])

        # LÓGICA PARA DEFINIR SE O ISQUEIRO FOI PERDIDO
        # Fatores que aumentam a probabilidade de perda:
        prob_perda = 0.2  # Base: 20%

        # Contexto de uso (rua, bar, festa = maior risco)
        if contexto_uso in ['Rua', 'Bar', 'Festa', 'Social']:
            prob_perda += 0.25
        elif contexto_uso in ['Parque', 'Carro']:
            prob_perda += 0.15
        elif contexto_uso in ['Trabalho']:
            prob_perda += 0.10
        # Casa mantém a probabilidade base

        # Perfil do fumante (sociais e esporádicos perdem mais)
        if perfil_fumante in ['Social', 'Esporadico']:
            prob_perda += 0.15
        elif perfil_fumante == 'Nao_fumante':
            prob_perda += 0.10

        # Tipo de isqueiro (descartáveis são perdidos mais facilmente)
        if tipo_isqueiro == 'Descartavel':
            prob_perda += 0.10

        # Idade (jovens perdem mais)
        if idade <= 25:
            prob_perda += 0.10
        elif idade <= 35:
            prob_perda += 0.05

        # Frequência alta de uso (usa muito = leva para muitos lugares = maior risco)
        if frequencia_uso > 3.0:
            prob_perda += 0.10

        # Gênero (ajuste sutil baseado em padrões comportamentais)
        if genero == 'M':
            prob_perda += 0.05

        # Limitar probabilidade entre 0 e 0.8 (máximo 80% de chance)
        prob_perda = min(0.8, max(0.05, prob_perda))

        # Decidir se perdeu ou não
        perdeu = np.random.random() < prob_perda

        # Se perdeu, definir data e local da perda
        if perdeu:
            dias_ate_perda = np.random.randint(1, tempo_posse + 1)
            data_perda = data_compra + timedelta(days=dias_ate_perda)

            # Local de perda baseado no contexto de uso
            locais_perda = {
                'Rua': ['Na_rua', 'Ponto_onibus', 'Calcada', 'Praca'],
                'Bar': ['Bar', 'Restaurante', 'Balada', 'Mesa_bar'],
                'Festa': ['Casa_amigo', 'Festa', 'Evento', 'Churrasco'],
                'Social': ['Encontro_amigos', 'Social', 'Reuniao'],
                'Trabalho': ['Escritorio', 'Trabalho', 'Cafeteria'],
                'Parque': ['Parque', 'Praca', 'Area_verde'],
                'Carro': ['Carro', 'Veiculo', 'Estacionamento'],
                'Casa': ['Casa', 'Quarto', 'Cozinha', 'Sala']
            }

            local_perda = np.random.choice(locais_perda.get(contexto_uso, ['Desconhecido']))
        else:
            dias_ate_perda = 0
            data_perda = None
            local_perda = 'N/A'

        # Mês/ano para análises temporais
        mes_ano = data_uso.strftime('%Y-%m')

        # Adicionar ao dataset
        registro = {
            'id_usuario': usuario_id,
            'nome': nome,
            'genero': genero,
            'idade': idade,
            'faixa_idade': faixa_idade,
            'perfil_fumante': perfil_fumante,
            'marca_isqueiro': marca,
            'cor_isqueiro': cor,
            'tipo_isqueiro': tipo_isqueiro,
            'preco_compra': round(preco_compra, 2),
            'data_compra': data_compra,
            'data_uso': data_uso,
            'tempo_posse': tempo_posse,
            'contexto_uso': contexto_uso,
            'proposito_uso': proposito_uso,
            'num_ascendimentos': num_ascendimentos,
            'frequencia_uso': round(frequencia_uso, 2),
            'mes_ano': mes_ano,
            'perdeu': perdeu,
            'dias_ate_perda': dias_ate_perda,
            'data_perda': data_perda,
            'local_perda': local_perda
        }

        dados.append(registro)

    # Criar DataFrame
    df = pd.DataFrame(dados)

    # Ajustar tipos de dados
    df['data_compra'] = pd.to_datetime(df['data_compra'])
    df['data_uso'] = pd.to_datetime(df['data_uso'])
    df['data_perda'] = pd.to_datetime(df['data_perda'])
    df['perdeu'] = df['perdeu'].astype(bool)

    print(f"✅ Base sintética gerada com sucesso!")
    print(f"📊 Total de registros: {len(df)}")
    print(f"🎯 Distribuição da target:")
    print(f"   - Perdidos: {df['perdeu'].sum()} ({df['perdeu'].mean()*100:.1f}%)")
    print(f"   - Não perdidos: {(~df['perdeu']).sum()} ({(1-df['perdeu'].mean())*100:.1f}%)")

    return df

# Gerar a base de dados
df = gerar_base_sintetica(n_registros=1200, seed=42)

# Salvar a base bruta
df.to_csv('lighter_tracker_dados_brutos.csv', index=False, encoding='utf-8')
print(f"\n💾 Base salva como 'lighter_tracker_dados_brutos.csv'")


🔄 Gerando 1200 registros sintéticos...
✅ Base sintética gerada com sucesso!
📊 Total de registros: 1200
🎯 Distribuição da target:
   - Perdidos: 606 (50.5%)
   - Não perdidos: 594 (49.5%)

💾 Base salva como 'lighter_tracker_dados_brutos.csv'


# 3. ANÁLISE EXPLORATÓRIA DOS DADOS (EDA)

In [3]:

print("\n" + "="*60)
print("📊 ANÁLISE EXPLORATÓRIA DOS DADOS")
print("="*60)

# Informações gerais sobre a base
print("\n🔍 INFORMAÇÕES GERAIS DA BASE:")
print(f"Dimensões: {df.shape}")
print(f"Período dos dados: {df['data_compra'].min().strftime('%Y-%m-%d')} a {df['data_uso'].max().strftime('%Y-%m-%d')}")

# Verificar valores nulos
print(f"\n🔍 VALORES NULOS:")
valores_nulos = df.isnull().sum()
print(valores_nulos[valores_nulos > 0])

# Estatísticas descritivas das variáveis numéricas
print(f"\n🔍 ESTATÍSTICAS DAS VARIÁVEIS NUMÉRICAS:")
print(df[['idade', 'preco_compra', 'tempo_posse', 'num_ascendimentos', 'frequencia_uso']].describe())

# Distribuição da variável target
print(f"\n🎯 DISTRIBUIÇÃO DA VARIÁVEL TARGET (perdeu):")
target_dist = df['perdeu'].value_counts()
print(target_dist)
print(f"Percentual de perdas: {df['perdeu'].mean()*100:.2f}%")

# Análise por contexto de uso
print(f"\n🏢 PERDAS POR CONTEXTO DE USO:")
perdas_contexto = df.groupby('contexto_uso')['perdeu'].agg(['count', 'sum', 'mean']).round(3)
perdas_contexto.columns = ['Total', 'Perdidos', 'Taxa_Perda']
perdas_contexto = perdas_contexto.sort_values('Taxa_Perda', ascending=False)
print(perdas_contexto)

# Análise por perfil de fumante
print(f"\n🚬 PERDAS POR PERFIL DE FUMANTE:")
perdas_perfil = df.groupby('perfil_fumante')['perdeu'].agg(['count', 'sum', 'mean']).round(3)
perdas_perfil.columns = ['Total', 'Perdidos', 'Taxa_Perda']
perdas_perfil = perdas_perfil.sort_values('Taxa_Perda', ascending=False)
print(perdas_perfil)

# Análise por marca
print(f"\n🏷️ PERDAS POR MARCA:")
perdas_marca = df.groupby('marca_isqueiro')['perdeu'].agg(['count', 'sum', 'mean']).round(3)
perdas_marca.columns = ['Total', 'Perdidos', 'Taxa_Perda']
perdas_marca = perdas_marca.sort_values('Taxa_Perda', ascending=False)
print(perdas_marca)


📊 ANÁLISE EXPLORATÓRIA DOS DADOS

🔍 INFORMAÇÕES GERAIS DA BASE:
Dimensões: (1200, 22)
Período dos dados: 2024-09-17 a 2026-02-22

🔍 VALORES NULOS:
data_perda    594
dtype: int64

🔍 ESTATÍSTICAS DAS VARIÁVEIS NUMÉRICAS:
             idade  preco_compra  tempo_posse  num_ascendimentos  \
count  1200.000000   1200.000000  1200.000000        1200.000000   
mean     40.797500     12.428958    75.285833         120.417500   
std      13.686402     16.904736    44.694822         102.153574   
min      18.000000      0.520000     5.000000           1.000000   
25%      29.000000      2.480000    40.000000          40.000000   
50%      41.000000      3.690000    72.000000          96.000000   
75%      52.000000      9.490000   103.000000         167.250000   
max      64.000000     51.500000   199.000000         591.000000   

       frequencia_uso  
count     1200.000000  
mean         1.585883  
std          0.834748  
min          0.080000  
25%          0.890000  
50%          1.520000  

# 4. PRÉ-PROCESSAMENTO DOS DADOS PARA MACHINE LEARNING

In [7]:

print("\n" + "="*60)
print("🔧 PRÉ-PROCESSAMENTO DOS DADOS")
print("="*60)

def preparar_dados_ml(df):
    """
    Prepara os dados para Machine Learning
    - Seleciona features relevantes
    - Trata valores categóricos
    - Cria features derivadas
    """

    print("🔧 Iniciando pré-processamento...")

    # Criar uma cópia para não modificar o original
    df_ml = df.copy()

    # Features que serão utilizadas no modelo
    features_numericas = [
        'idade',
        'preco_compra',
        'tempo_posse',
        'num_ascendimentos',
        'frequencia_uso'
    ]

    features_categoricas = [
        'genero',
        'faixa_idade',
        'perfil_fumante',
        'marca_isqueiro',
        'tipo_isqueiro',
        'contexto_uso',
        'proposito_uso'
    ]

    # Criar features derivadas
    print("🔧 Criando features derivadas...")

    # Feature: Intensidade de uso (ascendimentos por dia de posse)
    df_ml['intensidade_uso'] = df_ml['num_ascendimentos'] / df_ml['tempo_posse']
    df_ml['intensidade_uso'] = df_ml['intensidade_uso'].fillna(0)

    # Feature: Categoria de preço
    df_ml['categoria_preco'] = pd.cut(df_ml['preco_compra'],
                                    bins=[0, 5, 15, 100],
                                    labels=['Barato', 'Medio', 'Caro'])

    # Feature: Categoria de idade
    df_ml['categoria_idade'] = pd.cut(df_ml['idade'],
                                    bins=[17, 25, 35, 50, 70],
                                    labels=['Jovem', 'Adulto_Jovem', 'Adulto', 'Maduro'])

    # Feature: Mês da compra (sazonalidade)
    df_ml['mes_compra'] = df_ml['data_compra'].dt.month

    # Atualizar listas de features
    features_numericas.extend(['intensidade_uso', 'mes_compra'])
    features_categoricas.extend(['categoria_preco', 'categoria_idade'])

    # Selecionar todas as features para o modelo
    all_features = features_numericas + features_categoricas

    print(f"✅ Features numéricas ({len(features_numericas)}): {features_numericas}")
    print(f"✅ Features categóricas ({len(features_categoricas)}): {features_categoricas}")

    return df_ml, all_features, features_numericas, features_categoricas

# Preparar dados
df_ml, all_features, features_numericas, features_categoricas = preparar_dados_ml(df)

# Definir X e y
X = df_ml[all_features]
y = df_ml['perdeu']

print(f"\n🎯 Dataset final para ML:")
print(f"   - Shape: {X.shape}")
print(f"   - Target distribution: {y.value_counts().to_dict()}")



🔧 PRÉ-PROCESSAMENTO DOS DADOS
🔧 Iniciando pré-processamento...
🔧 Criando features derivadas...
✅ Features numéricas (7): ['idade', 'preco_compra', 'tempo_posse', 'num_ascendimentos', 'frequencia_uso', 'intensidade_uso', 'mes_compra']
✅ Features categóricas (9): ['genero', 'faixa_idade', 'perfil_fumante', 'marca_isqueiro', 'tipo_isqueiro', 'contexto_uso', 'proposito_uso', 'categoria_preco', 'categoria_idade']

🎯 Dataset final para ML:
   - Shape: (1200, 16)
   - Target distribution: {True: 606, False: 594}


# 5. DIVISÃO DOS DADOS (TREINO/TESTE)

## Bibliotecas de Machine Learning

In [8]:


from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (classification_report, roc_auc_score, confusion_matrix,
                           accuracy_score, precision_score, recall_score, f1_score, roc_curve)


In [9]:
print("\n🔀 Dividindo dados em treino e teste...")

# Divisão estratificada para manter proporção da target
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,           # 20% para teste
    stratify=y,              # Manter proporção da target
    random_state=42          # Reprodutibilidade
)

print(f"✅ Divisão concluída:")
print(f"   - Treino: {X_train.shape[0]} amostras ({y_train.mean()*100:.1f}% perdas)")
print(f"   - Teste: {X_test.shape[0]} amostras ({y_test.mean()*100:.1f}% perdas)")


🔀 Dividindo dados em treino e teste...
✅ Divisão concluída:
   - Treino: 960 amostras (50.5% perdas)
   - Teste: 240 amostras (50.4% perdas)


# 6. PIPELINE DE PRÉ-PROCESSAMENTO

In [10]:

print("\n🔧 Criando pipeline de pré-processamento...")

# Definir transformações para cada tipo de variável
preprocessor = ColumnTransformer(
    transformers=[
        # Variáveis numéricas: padronização (média=0, desvio=1)
        ('num', StandardScaler(), features_numericas),

        # Variáveis categóricas: One-Hot Encoding
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), features_categoricas)
    ],
    remainder='drop'  # Descartar colunas não especificadas
)

print("✅ Pipeline de pré-processamento criado:")
print(f"   - Padronização para: {features_numericas}")
print(f"   - One-Hot Encoding para: {features_categoricas}")



🔧 Criando pipeline de pré-processamento...
✅ Pipeline de pré-processamento criado:
   - Padronização para: ['idade', 'preco_compra', 'tempo_posse', 'num_ascendimentos', 'frequencia_uso', 'intensidade_uso', 'mes_compra']
   - One-Hot Encoding para: ['genero', 'faixa_idade', 'perfil_fumante', 'marca_isqueiro', 'tipo_isqueiro', 'contexto_uso', 'proposito_uso', 'categoria_preco', 'categoria_idade']


# 7. MODELO DE REGRESSÃO LOGÍSTICA


In [11]:

print("\n" + "="*60)
print("🤖 TREINAMENTO DO MODELO DE REGRESSÃO LOGÍSTICA")
print("="*60)

# Criar pipeline completo (pré-processamento + modelo)
pipeline_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        class_weight='balanced',    # Lidar com desbalanceamento
        random_state=42,           # Reprodutibilidade
        max_iter=1000             # Evitar warnings de convergência
    ))
])

print("🤖 Pipeline criado com:")
print("   - Pré-processamento automático")
print("   - Regressão Logística com class_weight='balanced'")
print("   - Tratamento automático de desbalanceamento")


🤖 TREINAMENTO DO MODELO DE REGRESSÃO LOGÍSTICA
🤖 Pipeline criado com:
   - Pré-processamento automático
   - Regressão Logística com class_weight='balanced'
   - Tratamento automático de desbalanceamento


# 8. OTIMIZAÇÃO DE HIPERPARÂMETROS

In [12]:

print("\n🎯 Otimizando hiperparâmetros com GridSearchCV...")

# Definir grade de hiperparâmetros para testar
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100],           # Força da regularização
    'classifier__penalty': ['l1', 'l2'],                 # Tipo de regularização
    'classifier__solver': ['liblinear', 'saga']          # Algoritmo de otimização
}

# GridSearchCV com validação cruzada
grid_search = GridSearchCV(
    pipeline_lr,
    param_grid,
    cv=5,                      # 5-fold cross-validation
    scoring='f1',              # Métrica para otimização (boa para desbalanceamento)
    n_jobs=-1,                 # Usar todos os cores
    verbose=1                  # Mostrar progresso
)

# Treinar com otimização
print("⏳ Treinando modelo com diferentes combinações de hiperparâmetros...")
grid_search.fit(X_train, y_train)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_

print(f"\n✅ Otimização concluída!")
print(f"🏆 Melhores hiperparâmetros: {grid_search.best_params_}")
print(f"📊 Melhor F1-Score (CV): {grid_search.best_score_:.4f}")


🎯 Otimizando hiperparâmetros com GridSearchCV...
⏳ Treinando modelo com diferentes combinações de hiperparâmetros...
Fitting 5 folds for each of 20 candidates, totalling 100 fits

✅ Otimização concluída!
🏆 Melhores hiperparâmetros: {'classifier__C': 10, 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear'}
📊 Melhor F1-Score (CV): 0.5665


# 9. AVALIAÇÃO DO MODELO

In [13]:

print("\n" + "="*60)
print("📊 AVALIAÇÃO DO MODELO")
print("="*60)

# Fazer predições no conjunto de teste
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilidades da classe positiva

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_proba)

print("🎯 MÉTRICAS DE AVALIAÇÃO:")
print(f"   - Acurácia:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   - Precisão:  {precision:.4f} ({precision*100:.2f}%)")
print(f"   - Recall:    {recall:.4f} ({recall*100:.2f}%)")
print(f"   - F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
print(f"   - AUC-ROC:   {auc_roc:.4f} ({auc_roc*100:.2f}%)")

print(f"\n📋 RELATÓRIO DETALHADO:")
print(classification_report(y_test, y_pred, target_names=['Não Perdeu', 'Perdeu']))

print(f"\n🔍 MATRIZ DE CONFUSÃO:")
cm = confusion_matrix(y_test, y_pred)
print(f"                 Predito")
print(f"              Não  |  Sim")
print(f"Real Não  [{cm[0,0]:4d}  |  {cm[0,1]:3d}]")
print(f"     Sim  [{cm[1,0]:4d}  |  {cm[1,1]:3d}]")


📊 AVALIAÇÃO DO MODELO
🎯 MÉTRICAS DE AVALIAÇÃO:
   - Acurácia:  0.5833 (58.33%)
   - Precisão:  0.5882 (58.82%)
   - Recall:    0.5785 (57.85%)
   - F1-Score:  0.5833 (58.33%)
   - AUC-ROC:   0.6222 (62.22%)

📋 RELATÓRIO DETALHADO:
              precision    recall  f1-score   support

  Não Perdeu       0.58      0.59      0.58       119
      Perdeu       0.59      0.58      0.58       121

    accuracy                           0.58       240
   macro avg       0.58      0.58      0.58       240
weighted avg       0.58      0.58      0.58       240


🔍 MATRIZ DE CONFUSÃO:
                 Predito
              Não  |  Sim
Real Não  [  70  |   49]
     Sim  [  51  |   70]


# 10. INTERPRETAÇÃO DO MODELO (FEATURE IMPORTANCE)

In [14]:

print("\n" + "="*60)
print("🔍 INTERPRETAÇÃO DO MODELO")
print("="*60)

# Obter nomes das features após pré-processamento
feature_names = (
    features_numericas +
    list(best_model.named_steps['preprocessor']
         .named_transformers_['cat']
         .get_feature_names_out(features_categoricas))
)

# Obter coeficientes do modelo
coeficientes = best_model.named_steps['classifier'].coef_[0]

# Criar DataFrame com feature importance
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Coeficiente': coeficientes,
    'Importancia_Abs': np.abs(coeficientes)
}).sort_values('Importancia_Abs', ascending=False)

print("🔍 TOP 15 FEATURES MAIS IMPORTANTES:")
print("(Coeficientes positivos aumentam probabilidade de perda)")
print("(Coeficientes negativos diminuem probabilidade de perda)")
print("-" * 60)
for idx, row in feature_importance.head(15).iterrows():
    sinal = "↑" if row['Coeficiente'] > 0 else "↓"
    print(f"{sinal} {row['Feature']:<35} | {row['Coeficiente']:>8.4f}")


🔍 INTERPRETAÇÃO DO MODELO
🔍 TOP 15 FEATURES MAIS IMPORTANTES:
(Coeficientes positivos aumentam probabilidade de perda)
(Coeficientes negativos diminuem probabilidade de perda)
------------------------------------------------------------
↓ marca_isqueiro_Zippo                |  -1.1393
↓ contexto_uso_Casa                   |  -1.0691
↓ contexto_uso_Trabalho               |  -1.0142
↓ contexto_uso_Carro                  |  -0.9783
↓ perfil_fumante_Pesado               |  -0.9483
↓ perfil_fumante_Moderado             |  -0.8240
↓ contexto_uso_Parque                 |  -0.6839
↓ proposito_uso_Charuto               |  -0.6680
↓ categoria_preco_Medio               |  -0.6534
↓ perfil_fumante_Social               |  -0.6162
↓ marca_isqueiro_Generico             |  -0.6098
↑ marca_isqueiro_Clipper              |   0.5646
↓ proposito_uso_Incenso               |  -0.5624
↑ categoria_idade_Jovem               |   0.5301
↓ proposito_uso_Tabaco                |  -0.4554


# 11. VALIDAÇÃO CRUZADA COMPLETA

In [15]:

print("\n🔄 Validação cruzada completa (5-fold)...")

# Validação cruzada com múltiplas métricas
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
cv_results = {}

for score in scoring:
    scores = cross_val_score(best_model, X, y, cv=5, scoring=score, n_jobs=-1)
    cv_results[score] = {
        'mean': scores.mean(),
        'std': scores.std(),
        'scores': scores
    }

print("📊 RESULTADOS DA VALIDAÇÃO CRUZADA (5-fold):")
for metric, results in cv_results.items():
    print(f"   - {metric.upper():<12}: {results['mean']:.4f} (±{results['std']:.4f})")


🔄 Validação cruzada completa (5-fold)...
📊 RESULTADOS DA VALIDAÇÃO CRUZADA (5-fold):
   - ACCURACY    : 0.5767 (±0.0341)
   - PRECISION   : 0.5843 (±0.0359)
   - RECALL      : 0.5694 (±0.0346)
   - F1          : 0.5761 (±0.0289)
   - ROC_AUC     : 0.6082 (±0.0403)


# 12. PREDIÇÕES NA BASE COMPLETA

In [16]:

print("\n🔮 Gerando predições para toda a base...")

# Fazer predições para toda a base de dados
X_full = df_ml[all_features]
y_full = df_ml['perdeu']

probabilidades_completas = best_model.predict_proba(X_full)[:, 1]
predicoes_completas = best_model.predict(X_full)

# Adicionar predições ao DataFrame original
df_final = df_ml.copy()
df_final['probabilidade_perda'] = probabilidades_completas
df_final['predicao_perda'] = predicoes_completas
df_final['acerto_modelo'] = (df_final['predicao_perda'] == df_final['perdeu'])

print(f"✅ Predições geradas para {len(df_final)} registros")
print(f"📊 Acurácia geral: {df_final['acerto_modelo'].mean()*100:.2f}%")

# Salvar base com predições
df_final.to_csv('lighter_tracker_com_predicoes.csv', index=False, encoding='utf-8')
print("💾 Base com predições salva como 'lighter_tracker_com_predicoes.csv'")


🔮 Gerando predições para toda a base...
✅ Predições geradas para 1200 registros
📊 Acurácia geral: 60.83%
💾 Base com predições salva como 'lighter_tracker_com_predicoes.csv'


# 13. ANÁLISE DE CASOS EXTREMOS

In [17]:

print("\n🔍 Analisando casos extremos...")

# Casos com maior probabilidade de perda
print("🔴 TOP 5 CASOS COM MAIOR RISCO DE PERDA:")
top_risco = df_final.nlargest(5, 'probabilidade_perda')[
    ['nome', 'contexto_uso', 'proposito_uso', 'perfil_fumante',
     'marca_isqueiro', 'probabilidade_perda', 'perdeu']
]
print(top_risco.to_string(index=False))

# Casos com menor probabilidade de perda
print("\n🟢 TOP 5 CASOS COM MENOR RISCO DE PERDA:")
baixo_risco = df_final.nsmallest(5, 'probabilidade_perda')[
    ['nome', 'contexto_uso', 'proposito_uso', 'perfil_fumante',
     'marca_isqueiro', 'probabilidade_perda', 'perdeu']
]
print(baixo_risco.to_string(index=False))


🔍 Analisando casos extremos...
🔴 TOP 5 CASOS COM MAIOR RISCO DE PERDA:
        nome contexto_uso proposito_uso perfil_fumante marca_isqueiro  probabilidade_perda  perdeu
Usuario_1128          Bar         Fogao     Esporadico         Ronson             0.935948    True
 Usuario_935          Rua          Vela     Esporadico         Ronson             0.872025    True
 Usuario_396          Rua         Fogao         Social         Ronson             0.869770    True
 Usuario_650          Rua       Cigarro     Esporadico         Ronson             0.858746    True
 Usuario_925          Rua          Vela     Esporadico            Bic             0.852782    True

🟢 TOP 5 CASOS COM MENOR RISCO DE PERDA:
        nome contexto_uso proposito_uso perfil_fumante marca_isqueiro  probabilidade_perda  perdeu
Usuario_1145         Casa        Tabaco    Nao_fumante          Zippo             0.188125   False
Usuario_1069         Casa       Incenso       Moderado       Generico             0.192565   Fa

# 14. PREPARAÇÃO PARA DEPLOY

## Para deploy do modelo


In [18]:
import pickle
import joblib
import json


In [19]:

print("\n" + "="*60)
print("🚀 PREPARAÇÃO PARA DEPLOY")
print("="*60)

# Salvar o modelo treinado
model_filename = 'lighter_tracker_model.pkl'
joblib.dump(best_model, model_filename)
print(f"💾 Modelo salvo como '{model_filename}'")

# Salvar informações do modelo
model_info = {
    'model_type': 'Logistic Regression',
    'features': all_features,
    'features_numericas': features_numericas,
    'features_categoricas': features_categoricas,
    'target': 'perdeu',
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'auc_roc': auc_roc,
    'best_params': grid_search.best_params_,
    'cv_f1_mean': cv_results['f1']['mean'],
    'training_date': datetime.now().isoformat(),
    'total_samples': len(df_final),
    'positive_class_ratio': y.mean()
}

with open('model_info.json', 'w', encoding='utf-8') as f:
    json.dump(model_info, f, indent=2, ensure_ascii=False)

print("💾 Informações do modelo salvas como 'model_info.json'")

# Criar função para predição em produção
def criar_funcao_predicao():
    """
    Cria função para ser usada em produção
    """
    codigo_producao = '''
import joblib
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

def carregar_modelo():
    """Carrega o modelo treinado"""
    try:
        model = joblib.load('lighter_tracker_model.pkl')
        return model
    except Exception as e:
        raise Exception(f"Erro ao carregar modelo: {e}")

def preparar_entrada(dados_usuario):
    """
    Prepara os dados de entrada para predição

    Parâmetros esperados no dicionário dados_usuario:
    - genero: 'M' ou 'F'
    - idade: int (18-65)
    - perfil_fumante: 'Pesado', 'Moderado', 'Social', 'Esporadico', 'Nao_fumante'
    - marca_isqueiro: 'Bic', 'Zippo', 'Clipper', 'Cricket', 'Ronson', 'Generico'
    - tipo_isqueiro: 'Descartavel' ou 'Recarregavel'
    - preco_compra: float (0.5-100.0)
    - tempo_posse: int (dias)
    - contexto_uso: 'Casa', 'Rua', 'Trabalho', 'Bar', 'Parque', 'Social', 'Festa', 'Carro'
    - proposito_uso: 'Cigarro', 'Tabaco', 'Vela', 'Incenso', 'Fogao', 'Camping', 'Charuto', 'Outro'
    - num_ascendimentos: int
    - frequencia_uso: float (ascendimentos por dia)
    """

    # Criar features derivadas
    intensidade_uso = dados_usuario['num_ascendimentos'] / max(dados_usuario['tempo_posse'], 1)

    # Categoria de preço
    if dados_usuario['preco_compra'] <= 5:
        categoria_preco = 'Barato'
    elif dados_usuario['preco_compra'] <= 15:
        categoria_preco = 'Medio'
    else:
        categoria_preco = 'Caro'

    # Categoria de idade
    idade = dados_usuario['idade']
    if idade <= 25:
        categoria_idade = 'Jovem'
    elif idade <= 35:
        categoria_idade = 'Adulto_Jovem'
    elif idade <= 50:
        categoria_idade = 'Adulto'
    else:
        categoria_idade = 'Maduro'

    # Faixa etária
    if idade <= 25:
        faixa_idade = '18-25'
    elif idade <= 35:
        faixa_idade = '26-35'
    elif idade <= 50:
        faixa_idade = '36-50'
    else:
        faixa_idade = '51+'

    # Mês atual
    mes_compra = datetime.now().month

    # Criar DataFrame com todas as features necessárias
    features_dict = {
        'idade': dados_usuario['idade'],
        'preco_compra': dados_usuario['preco_compra'],
        'tempo_posse': dados_usuario['tempo_posse'],
        'num_ascendimentos': dados_usuario['num_ascendimentos'],
        'frequencia_uso': dados_usuario['frequencia_uso'],
        'intensidade_uso': intensidade_uso,
        'mes_compra': mes_compra,
        'genero': dados_usuario['genero'],
        'faixa_idade': faixa_idade,
        'perfil_fumante': dados_usuario['perfil_fumante'],
        'marca_isqueiro': dados_usuario['marca_isqueiro'],
        'tipo_isqueiro': dados_usuario['tipo_isqueiro'],
        'contexto_uso': dados_usuario['contexto_uso'],
        'proposito_uso': dados_usuario['proposito_uso'],
        'categoria_preco': categoria_preco,
        'categoria_idade': categoria_idade
    }

    return pd.DataFrame([features_dict])

def prever_perda_isqueiro(dados_usuario):
    """
    Função principal para prever perda de isqueiro

    Retorna:
    - probabilidade: float (0-1) - Probabilidade de perda
    - categoria_risco: str - Categoria do risco
    - recomendacao: str - Recomendação para o usuário
    """

    try:
        # Carregar modelo
        model = carregar_modelo()

        # Preparar dados
        df_input = preparar_entrada(dados_usuario)

        # Fazer predição
        probabilidade = model.predict_proba(df_input)[0, 1]
        predicao = model.predict(df_input)[0]

        # Categorizar risco
        if probabilidade < 0.3:
            categoria_risco = "Baixo"
            emoji_risco = "🟢"
        elif probabilidade < 0.6:
            categoria_risco = "Médio"
            emoji_risco = "🟡"
        else:
            categoria_risco = "Alto"
            emoji_risco = "🔴"

        # Gerar recomendações baseadas nos fatores de risco
        recomendacoes = []

        if dados_usuario['contexto_uso'] in ['Rua', 'Bar', 'Festa', 'Social']:
            recomendacoes.append("⚠️ Evite levar seu isqueiro para locais públicos se possível")

        if dados_usuario['perfil_fumante'] in ['Social', 'Esporadico']:
            recomendacoes.append("💡 Considere manter seu isqueiro sempre no mesmo bolso")

        if dados_usuario['tipo_isqueiro'] == 'Descartavel':
            recomendacoes.append("🔄 Isqueiros descartáveis são perdidos mais facilmente - considere um recarregável")

        if dados_usuario['frequencia_uso'] > 3.0:
            recomendacoes.append("📱 Alto uso = alto risco. Configure lembretes para verificar seu isqueiro")

        if dados_usuario['idade'] <= 25:
            recomendacoes.append("👤 Jovens tendem a perder mais - tenha atenção extra!")

        if not recomendacoes:
            recomendacoes.append("✅ Continue com seus hábitos atuais - baixo risco de perda!")

        resultado = {
            'probabilidade': round(probabilidade * 100, 2),
            'categoria_risco': categoria_risco,
            'emoji_risco': emoji_risco,
            'predicao_binaria': 'Sim' if predicao else 'Não',
            'recomendacoes': recomendacoes,
            'fatores_risco': {
                'contexto_uso': dados_usuario['contexto_uso'],
                'perfil_fumante': dados_usuario['perfil_fumante'],
                'tipo_isqueiro': dados_usuario['tipo_isqueiro'],
                'frequencia_uso': dados_usuario['frequencia_uso'],
                'idade': dados_usuario['idade']
            }
        }

        return resultado

    except Exception as e:
        return {
            'erro': f"Erro na predição: {str(e)}",
            'probabilidade': None,
            'categoria_risco': None,
            'recomendacao': "Erro no processamento"
        }

# EXEMPLO DE USO:
if __name__ == "__main__":
    # Exemplo 1: Alto risco
    exemplo_alto_risco = {
        'genero': 'M',
        'idade': 22,
        'perfil_fumante': 'Social',
        'marca_isqueiro': 'Bic',
        'tipo_isqueiro': 'Descartavel',
        'preco_compra': 2.50,
        'tempo_posse': 15,
        'contexto_uso': 'Bar',
        'proposito_uso': 'Cigarro',
        'num_ascendimentos': 50,
        'frequencia_uso': 3.33
    }

    # Exemplo 2: Baixo risco
    exemplo_baixo_risco = {
        'genero': 'F',
        'idade': 45,
        'perfil_fumante': 'Nao_fumante',
        'marca_isqueiro': 'Zippo',
        'tipo_isqueiro': 'Recarregavel',
        'preco_compra': 55.00,
        'tempo_posse': 90,
        'contexto_uso': 'Casa',
        'proposito_uso': 'Vela',
        'num_ascendimentos': 25,
        'frequencia_uso': 0.28
    }

    print("=== TESTE DO MODELO ===")
    print("\\n🔴 EXEMPLO ALTO RISCO:")
    resultado1 = prever_perda_isqueiro(exemplo_alto_risco)
    print(f"Probabilidade: {resultado1['probabilidade']:.1f}%")
    print(f"Categoria: {resultado1['emoji_risco']} {resultado1['categoria_risco']}")
    print("Recomendações:")
    for rec in resultado1['recomendacoes']:
        print(f"  {rec}")

    print("\\n🟢 EXEMPLO BAIXO RISCO:")
    resultado2 = prever_perda_isqueiro(exemplo_baixo_risco)
    print(f"Probabilidade: {resultado2['probabilidade']:.1f}%")
    print(f"Categoria: {resultado2['emoji_risco']} {resultado2['categoria_risco']}")
    print("Recomendações:")
    for rec in resultado2['recomendacoes']:
        print(f"  {rec}")
'''

    return codigo_producao

# Salvar código de produção
codigo_producao = criar_funcao_predicao()
with open('lighter_tracker_predicao.py', 'w', encoding='utf-8') as f:
    f.write(codigo_producao)

print("💾 Código para produção salvo como 'lighter_tracker_predicao.py'")


🚀 PREPARAÇÃO PARA DEPLOY
💾 Modelo salvo como 'lighter_tracker_model.pkl'
💾 Informações do modelo salvas como 'model_info.json'
💾 Código para produção salvo como 'lighter_tracker_predicao.py'


# 15. TESTE DA FUNÇÃO DE PRODUÇÃO

In [21]:

print("\n🧪 Testando função de produção...")

# Simular exemplos para teste
exemplo_alto_risco = {
    'genero': 'M',
    'idade': 22,
    'perfil_fumante': 'Social',
    'marca_isqueiro': 'Bic',
    'tipo_isqueiro': 'Descartavel',
    'preco_compra': 2.50,
    'tempo_posse': 15,
    'contexto_uso': 'Bar',
    'proposito_uso': 'Cigarro',
    'num_ascendimentos': 50,
    'frequencia_uso': 3.33
}

exemplo_baixo_risco = {
    'genero': 'F',
    'idade': 45,
    'perfil_fumante': 'Nao_fumante',
    'marca_isqueiro': 'Zippo',
    'tipo_isqueiro': 'Recarregavel',
    'preco_compra': 55.00,
    'tempo_posse': 90,
    'contexto_uso': 'Casa',
    'proposito_uso': 'Vela',
    'num_ascendimentos': 25,
    'frequencia_uso': 0.28
}

# Função de teste (simulação da função de produção)
def testar_predicao(dados_usuario, nome_caso):
    """Testa a predição com os dados fornecidos"""

    # Preparar dados igual à função de produção
    df_test = pd.DataFrame([dados_usuario])

    # Criar features derivadas
    df_test['intensidade_uso'] = df_test['num_ascendimentos'] / df_test['tempo_posse']

    # Categorias
    df_test['categoria_preco'] = pd.cut(df_test['preco_compra'],
                                       bins=[0, 5, 15, 100],
                                       labels=['Barato', 'Medio', 'Caro'])

    df_test['categoria_idade'] = pd.cut(df_test['idade'],
                                       bins=[17, 25, 35, 50, 70],
                                       labels=['Jovem', 'Adulto_Jovem', 'Adulto', 'Maduro'])

    # Faixa etária
    idade = df_test['idade'].iloc[0]
    if idade <= 25:
        faixa_idade = '18-25'
    elif idade <= 35:
        faixa_idade = '26-35'
    elif idade <= 50:
        faixa_idade = '36-50'
    else:
        faixa_idade = '51+'

    df_test['faixa_idade'] = faixa_idade
    df_test['mes_compra'] = datetime.now().month

    # Selecionar features na ordem correta
    df_test = df_test[all_features]

    # Fazer predição
    probabilidade = best_model.predict_proba(df_test)[0, 1]

    # Categorizar risco
    if probabilidade < 0.3:
        categoria_risco = "Baixo 🟢"
    elif probabilidade < 0.6:
        categoria_risco = "Médio 🟡"
    else:
        categoria_risco = "Alto 🔴"

    print(f"\n{nome_caso}:")
    print(f"  Probabilidade de perda: {probabilidade*100:.1f}%")
    print(f"  Categoria de risco: {categoria_risco}")

    return probabilidade

# Testar exemplos
prob_alto = testar_predicao(exemplo_alto_risco, "🔴 EXEMPLO ALTO RISCO")
prob_baixo = testar_predicao(exemplo_baixo_risco, "🟢 EXEMPLO BAIXO RISCO")


🧪 Testando função de produção...

🔴 EXEMPLO ALTO RISCO:
  Probabilidade de perda: 81.0%
  Categoria de risco: Alto 🔴

🟢 EXEMPLO BAIXO RISCO:
  Probabilidade de perda: 29.8%
  Categoria de risco: Baixo 🟢


# 16. RESUMO FINAL E MÉTRICAS DO PROJETO


In [22]:

print("\n" + "="*60)
print("📋 RESUMO FINAL DO PROJETO")
print("="*60)

print(f"""
🎯 PROJETO LIGHTER TRACKER - SISTEMA DE PREDIÇÃO DE PERDA DE ISQUEIROS
================================================================

📊 DADOS:
   • Base sintética gerada: {len(df)} registros
   • Período: {df['data_compra'].min().strftime('%Y-%m-%d')} a {df['data_uso'].max().strftime('%Y-%m-%d')}
   • Taxa de perda real: {df['perdeu'].mean()*100:.1f}%
   • Features utilizadas: {len(all_features)} ({len(features_numericas)} numéricas + {len(features_categoricas)} categóricas)

🤖 MODELO:
   • Algoritmo: Regressão Logística
   • Pré-processamento: StandardScaler + OneHotEncoder
   • Balanceamento: class_weight='balanced'
   • Otimização: GridSearchCV (5-fold CV)

📈 PERFORMANCE:
   • Acurácia:     {accuracy*100:.2f}%
   • Precisão:     {precision*100:.2f}%
   • Recall:       {recall*100:.2f}%
   • F1-Score:     {f1*100:.2f}%
   • AUC-ROC:      {auc_roc*100:.2f}%

🔍 PRINCIPAIS INSIGHTS:
   • Contextos de maior risco: Rua, Bar, Festa, Social
   • Perfis de maior risco: Social, Esporádico
   • Isqueiros descartáveis são perdidos mais facilmente
   • Jovens (18-25) têm maior tendência à perda
   • Uso frequente aumenta risco (mais lugares = mais risco)

📁 ARQUIVOS GERADOS:
   ✅ lighter_tracker_dados_brutos.csv - Base sintética original
   ✅ lighter_tracker_com_predicoes.csv - Base com predições
   ✅ lighter_tracker_model.pkl - Modelo treinado para deploy
   ✅ model_info.json - Informações e métricas do modelo
   ✅ lighter_tracker_predicao.py - Código para produção

🚀 APLICAÇÃO PRÁTICA:
   • Sistema de recomendações em tempo real
   • Alertas preventivos baseados em comportamento
   • Insights para melhorar retenção de isqueiros
   • Interface amigável com categorização de risco

🔮 EXEMPLO DE PREDIÇÃO:
   • Alto risco (jovem, bar, descartável): {prob_alto*100:.1f}%
   • Baixo risco (adulto, casa, recarregável): {prob_baixo*100:.1f}%

================================================================
✅ PROJETO CONCLUÍDO COM SUCESSO!
💡 Pronto para integração em sistema web/mobile
📱 Código otimizado para deploy em produção
""")

print("\n🎉 FIM DO NOTEBOOK - LIGHTER TRACKER ML PIPELINE")
print("=" * 60)


📋 RESUMO FINAL DO PROJETO

🎯 PROJETO LIGHTER TRACKER - SISTEMA DE PREDIÇÃO DE PERDA DE ISQUEIROS

📊 DADOS:
   • Base sintética gerada: 1200 registros
   • Período: 2024-09-17 a 2026-02-22
   • Taxa de perda real: 50.5%
   • Features utilizadas: 16 (7 numéricas + 9 categóricas)

🤖 MODELO:
   • Algoritmo: Regressão Logística
   • Pré-processamento: StandardScaler + OneHotEncoder
   • Balanceamento: class_weight='balanced'
   • Otimização: GridSearchCV (5-fold CV)

📈 PERFORMANCE:
   • Acurácia:     58.33%
   • Precisão:     58.82%
   • Recall:       57.85%
   • F1-Score:     58.33%
   • AUC-ROC:      62.22%

🔍 PRINCIPAIS INSIGHTS:
   • Contextos de maior risco: Rua, Bar, Festa, Social
   • Perfis de maior risco: Social, Esporádico
   • Isqueiros descartáveis são perdidos mais facilmente
   • Jovens (18-25) têm maior tendência à perda
   • Uso frequente aumenta risco (mais lugares = mais risco)

📁 ARQUIVOS GERADOS:
   ✅ lighter_tracker_dados_brutos.csv - Base sintética original
   ✅ light